In [1]:
import os
import random
import cv2 # OpenCV для работы с изображениями
import re
import requests

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models import resnet18, ResNet18_Weights # Наша предобученная модель

from collections import defaultdict # Удобный словарь, который не выдает ошибку, если ключа нет
import matplotlib.pyplot as plt

In [ ]:
#https://www.kaggle.com/code/ayberkural/autoprice-ai-uncovering-car-value-insights

In [2]:
# Рекомендуем запускать код в среде выполения с cuda

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [3]:
def get_direct_file_link(mailru_file_url: str) -> str:
    """
    Превращает ссылку на конкретный файл внутри папки cloud.mail.ru/public/XXX/YYYY/filename
    в прямую ссылку на CDN.
    """
    resp = requests.get(mailru_file_url)
    if resp.status_code != 200:
        raise RuntimeError(f"Ошибка {resp.status_code} при запросе {mailru_file_url}")
    page = resp.text

    match = re.search(r'dispatcher.*?weblink_get.*?url":"(.*?)"', page)
    if not match:
        raise RuntimeError("Не удалось найти CDN ссылку в HTML")
    base_url = match.group(1)

    # вычленяем /XXX/YYYY/filename
    parts = mailru_file_url.split('/')[-3:]
    return f"{base_url}/{parts[0]}/{parts[1]}/{parts[2]}"

def download_from_mailru(file_url: str, local_name: str):
    direct = get_direct_file_link(file_url)
    print(f"Скачиваем {file_url} → {local_name}")
    os.system(f"wget --content-disposition '{direct}' -O '{local_name}'")

In [ ]:
# Ссылки на архивы с картинками
train_link = "https://cloud.mail.ru/public/2kaD/W4xWY9vgr/train_images.zip"
test_link  = "https://cloud.mail.ru/public/2kaD/W4xWY9vgr/test_images.zip"

train_zip = download_from_mailru(train_link, "train_images.zip")
test_zip  = download_from_mailru(test_link,  "test_images.zip")

In [ ]:
# Распаковываем архивы в соответствующие папки

!unzip -q train_images.zip -d train_images
!unzip -q test_images.zip -d test_images

In [ ]:
TRAIN_PQ = "/kaggle/input/task-2-autoprice-price-prediction-based-on-photos/train_dataset.parquet"
TEST_PQ  = "/kaggle/input/task-2-autoprice-price-prediction-based-on-photos/test_dataset.parquet"
TRAIN_IMG_DIR = "train_images"
TEST_IMG_DIR  = "test_images"

TRAIN_CSV = "train.csv"
TEST_CSV  = "test.csv"
CKPT_NAME = "resne18_IMG192"
CKPT_PATH = f"{CKPT_NAME}.pth"
USE_LOG_TARGET = True # Используем логарифмический маштаб таргета

In [ ]:
# Гиперпараметры

IMG_SIZE = 192         # Приводим все картинки к размеру 192x192
N_IMAGES_PER_ITEM = 1  # Для бейзлайна берем только 1 картинку на объявление
BATCH_SIZE = 16        # Сколько картинок обрабатывать за один шаг
EPOCHS = 5             # Сколько раз прогонять все обучающие данные
LR = 1e-4              # Скорость обучения (learning rate)
WEIGHT_DECAY = 1e-5    # Параметр регуляризации для предотвращения переобучения
VAL_SIZE = 0.2         # Какую долю данных отложить для валидации (20%)
KEEP_RATIO = 1     # Для ускорения экспериментов можем взять не все данные, а только 50%

# Фиксируем "зерно" случайности, чтобы наши эксперименты были повторяемыми
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
import pandas as pd

In [135]:
# Загружаем табличные данные
train_df = pd.read_parquet(TRAIN_PQ)
test_df  = pd.read_parquet(TEST_PQ)

In [ ]:
y = model.predict(test[feature_columns])

s = pd.read_csv('/kaggle/input/task-2-autoprice-price-prediction-based-on-photos/sample_submission.csv')

s.target = np.exp(y)
s

In [ ]:
max(a)

In [ ]:
a = []

In [ ]:
for r in train_df.itertuples(index=False):
        iid = getattr(r, "ID")
        paths = sorted(id_to_files.get(iid, []))
        a.append(len(paths))
    

In [ ]:
def build_img_csv(df, img_dir, out_csv, n_images=1, keep_ratio=1.0, seed=42):

    """
    Функция для создания CSV-файла, связывающего ID объявления с путями к его фотографиям.
    """

    all_files = os.listdir(img_dir)
    id_to_files = defaultdict(list)
    for fname in all_files:
        if fname.endswith(".jpg"):
            iid = int(fname.split("_")[0])  # предполагаем формат ID_xxx.jpg
            id_to_files[iid].append(os.path.join(img_dir, fname))

    rows = []
    for r in df.itertuples(index=False):
        iid = getattr(r, "ID")
        paths = sorted(id_to_files.get(iid, []))
        if len(paths) > n_images:
            paths = paths[:n_images]

        row = {
            "item_id": iid,
            "paths": ";".join(paths)
        }

        # цена есть только в train
        if "price_TARGET" in df.columns:
            row["price"] = getattr(r, "price_TARGET")
        else:
            row["price"] = -1  # заглушка для test

        rows.append(row)

    # Создаем и сохраняем итоговый DataFrame
    csv_df = pd.DataFrame(rows)

    # Если нужно, оставляем только часть данных для быстрых экспериментов
    if keep_ratio < 1.0:
        csv_df = csv_df.sample(frac=keep_ratio, random_state=seed).reset_index(drop=True)

    csv_df.to_csv(out_csv, index=False)
    print(f"CSV сохранён: {out_csv}, shape={csv_df.shape}")
    return csv_df

In [ ]:
# Запускаем функцию для train и test выборок

train_csv_df = build_img_csv(train_df, TRAIN_IMG_DIR, TRAIN_CSV)
test_csv_df  = build_img_csv(test_df,  TEST_IMG_DIR,  TEST_CSV)

In [ ]:
# Реализуем функцию для подсчета medianAPE в точности как в правилах соревнования

def median_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true > 0
    ape = np.abs(y_true[mask] - y_pred[mask]) / (y_true[mask] + 1e-6)
    return float(np.median(ape))

In [ ]:
# Создаем конвейер преобразований для ТРЕНИРОВОЧНЫХ данных

train_tfms = A.Compose([
    # Шаги предобработки (обязательные)
    A.LongestMaxSize(IMG_SIZE), # Уменьшаем картинку по длинной стороне до IMG_SIZE
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_CONSTANT), # Добавляем поля до квадрата
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Нормализуем "магическими" числами ImageNet

    # Шаги аугментации (случайные)
    A.HorizontalFlip(p=0.5), # Отражаем по горизонтали с вероятностью 50%
    A.RandomBrightnessContrast(p=0.3), # Меняем яркость и контраст
    A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.1, rotate_limit=10,
                       border_mode=cv2.BORDER_CONSTANT, p=0.5), # Сдвигаем, масштабируем и поворачиваем

    # Финальный шаг предобработки
    ToTensorV2(), # Превращаем numpy-массив в тензор PyTorch
])

# Создаем конвейер преобразований для ВАЛИДАЦИОННЫХ данных (только обязательные шаги)
valid_tfms = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_CONSTANT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
def _safe_imread(p: str, fallback_hw=(IMG_SIZE, IMG_SIZE)) -> np.ndarray:
    """Безопасно читает картинку. Если файл битый или не существует, возвращает черный квадрат."""
    img = cv2.imread(p, cv2.IMREAD_COLOR)
    if img is None:
        # Если картинка не прочиталась, создаем "пустышку"
        h, w = fallback_hw
        img = np.zeros((h, w, 3), dtype=np.uint8)
    else:
        # OpenCV читает картинки в формате BGR, а нам нужен RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


class CarsDataset(Dataset):
    """
    Класс датасета для PyTorch.
    """
    def __init__(self, df: pd.DataFrame, is_train: bool = True):
        self.df = df.reset_index(drop=True)
        # Выбираем нужный набор аугментаций в зависимости от того, обучающий это датасет или нет
        self.tfms = train_tfms if is_train else valid_tfms
        self.has_target = "price" in df.columns

    def __len__(self):
        """Возвращает общее количество объектов в датасете."""
        return len(self.df)

    def __getitem__(self, idx: int):
        """По индексу `idx` возвращает один готовый для обучения пример."""
        row = self.df.iloc[idx]
        # Берем пути к картинкам из строки и очищаем от возможных пустых значений
        paths = [p for p in str(row["paths"]).split(";") if p and p.lower() != "nan"][:N_IMAGES_PER_ITEM]

        imgs = []
        for p in paths:
            img = _safe_imread(p) # Читаем картинку
            img = self.tfms(image=img)["image"].contiguous() # Применяем аугментации
            imgs.append(img)

        # Если картинок меньше, чем N_IMAGES_PER_ITEM, добиваем "пустыми" тензорами
        while len(imgs) < N_IMAGES_PER_ITEM:
            imgs.append(torch.zeros(3, IMG_SIZE, IMG_SIZE))

        # Собираем список тензоров в один тензор формы (N, C, H, W)
        imgs = torch.stack(imgs, dim=0)

        # Если это обучающие данные, возвращаем картинки и цену
        if self.has_target:
            y = torch.tensor(row["price"], dtype=torch.float32)
            # Если включен флаг, берем логарифм от цены. Это стабилизирует обучение.
            if USE_LOG_TARGET:
                y = torch.log1p(y)
            return imgs, y
        # Если это тестовые данные, возвращаем картинки и ID объявления
        else:
            return imgs, torch.tensor(row.get("item_id", -1), dtype=torch.long)

In [ ]:
def collate_fn(batch):
    """
    Вспомогательная функция для DataLoader. Она умеет правильно собирать
    пары (картинка, цена) или (картинка, ID) в один батч.
    """
    imgs = torch.stack([b[0] for b in batch], dim=0)  # (B, N, C, H, W)
    target = batch[0][1]

    # Если таргет - это число с плавающей точкой (цена), собираем тензор цен
    if torch.is_tensor(target) and target.dtype.is_floating_point:
        y = torch.stack([b[1] for b in batch], dim=0)  # (B,)
        return imgs, y
    # Иначе это ID, собираем тензор ID
    else:
        ids = torch.stack([b[1] for b in batch], dim=0)
        return imgs, ids

class ResNetRegressor(nn.Module):
    """
    Наша модель: "тушка" ResNet18 + "голова" для регрессии.
    """
    def __init__(self):
        super().__init__()
        # Загружаем ResNet18 с весами, предобученными на ImageNet
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features = backbone.fc.in_features # Узнаем размер эмбеддинга (у ResNet18 это 512)
        # "Отрезаем" последний слой (fc)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        # Создаем нашу "голову" - линейный слой, который из 512 признаков делает 1 число (цену)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x имеет форму (Batch, N_images, Channels, Height, Width)
        B, N, C, H, W = x.shape
        # "Распрямляем" батч, чтобы обработать все картинки одним прогоном
        x = x.view(B*N, C, H, W)
        # Получаем эмбеддинги (векторы признаков)
        feats = self.backbone(x)          # (B*N, 512, 1, 1)
        # Возвращаем эмбеддингам форму, связанную с батчем
        feats = feats.view(B, N, -1)      # (B, N, 512)
        # Усредняем эмбеддинги по всем картинкам одного объявления
        feats = feats.mean(dim=1)
        # Прогоняем усредненный эмбеддинг через "голову"
        out = self.head(feats)            # (B, 1)
        return out.squeeze(1)             # (B,) - возвращаем вектор предсказаний

In [ ]:
def run_epoch(model, loader, optimizer=None, device="cuda"):
    """Прогоняет одну эпоху обучения или валидации."""
    is_train = optimizer is not None  # Если передан optimizer - это обучение
    model.train(is_train) # Переключаем модель в режим train или eval

    losses = []
    preds_log_all, y_log_all = [], []

    # tqdm оборачивает итератор, чтобы показывать красивую полоску прогресса
    pbar = tqdm(loader, desc="Train" if is_train else "Valid", leave=False)
    for imgs, y_log in pbar:
        # Переносим данные на GPU
        imgs = imgs.to(device, non_blocking=True)
        y_log = y_log.view(-1).to(device)

        # Получаем предсказания модели
        preds_log = model(imgs)
        # Считаем ошибку. L1Loss (MAE) хорошо работает для регрессии, особенно с логарифмом.
        loss = nn.L1Loss()(preds_log, y_log)

        if is_train:
            optimizer.zero_grad(set_to_none=True) # Обнуляем градиенты
            loss.backward()                       # Считаем градиенты
            optimizer.step()                      # Обновляем веса модели

        # Собираем статистику
        losses.append(loss.item())
        preds_log_all.append(preds_log.detach().cpu())
        y_log_all.append(y_log.detach().cpu())
        pbar.set_postfix({"loss": f"{loss.item():.3f}"}) # Показываем текущую ошибку в прогресс-баре

    # Считаем итоговые метрики за всю эпоху
    preds_log_all = torch.cat(preds_log_all).numpy()
    y_log_all     = torch.cat(y_log_all).numpy()

    # Если использовали логарифм, возвращаем предсказания и таргет в исходный масштаб
    preds  = np.expm1(preds_log_all)
    y_true = np.expm1(y_log_all)

    medape = median_absolute_percentage_error(y_true, preds) * 100.0
    return float(medape), float(np.mean(losses))

In [ ]:
history = {
    "epoch": [],
    "tr_loss": [],
    "tr_medape": [],
    "val_loss": [],
    "val_medape": [],
}

In [ ]:
df = pd.read_csv(TRAIN_CSV)
trn_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED, shuffle=True)

train_ds = CarsDataset(trn_df, is_train=True)
valid_ds = CarsDataset(val_df, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          pin_memory=True, num_workers=4, drop_last=True,
                          collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False,
                          pin_memory=True, num_workers=4, drop_last=False,
                          collate_fn=collate_fn)

model = ResNetRegressor().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

best_val_medape = float("inf")

# Главный цикл обучения
for epoch in range(1, EPOCHS + 1):
    print(f"\nЭпоха {epoch}/{EPOCHS}")
    # Запускаем эпоху обучения
    tr_medape, tr_loss = run_epoch(model, train_loader, optimizer, device=device)
    # Запускаем эпоху валидации (optimizer=None)
    val_medape, val_loss = run_epoch(model, valid_loader, optimizer=None, device=device)

    print(f"Train loss: {tr_loss:.3f} | medianAPE: {tr_medape:.2f}%")
    print(f"Valid loss: {val_loss:.3f} | medianAPE: {val_medape:.2f}%")

    # Сохраняем историю
    history["epoch"].append(epoch)
    history["tr_loss"].append(tr_loss)
    history["tr_medape"].append(tr_medape)
    history["val_loss"].append(val_loss)
    history["val_medape"].append(val_medape)

    # Сохраняем модель (чекпоинт), только если она показала лучший результат на валидации
    if val_medape < best_val_medape:
        best_val_medape = val_medape
        torch.save({
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "epoch": epoch,
            "val_medape": val_medape
        }, CKPT_PATH)
        print(f"✓ Модель сохранена в эпоху {epoch} -> {CKPT_PATH}")

In [ ]:
# Визуализация истории обучения

plt.figure()
plt.plot(history["epoch"], history["tr_medape"], label="train medAPE")
plt.plot(history["epoch"], history["val_medape"], label="valid medAPE")
plt.xlabel("Epoch")
plt.ylabel("Median APE, %")
plt.title("Median APE vs Epoch")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"medape_vs_epoch_{CKPT_NAME}.png", dpi=150)
plt.show()


In [ ]:
# Сохраняем историю в CSV для дальнейшего анализа

csv_name = f"{CKPT_NAME}.csv"
df_hist = pd.DataFrame(history)
df_hist.to_csv(csv_name, index=False)

In [ ]:


#cb
import numpy as np

def create_dummy_columns(df):
    """
    Создает dummy-колонки для колонок со списками строк,
    игнорируя значения None в списках.
    
    Parameters:
    df (pd.DataFrame): Исходный DataFrame
    
    Returns:
    pd.DataFrame: DataFrame с добавленными dummy-колонками
    """
    df_result = df.copy()
    d_cols= []
    for column in df.columns:
        # Проверяем, что в колонке есть списки
        if df[column].apply(lambda x: isinstance(x, np.ndarray) ).any() or column in ['aktivnaya_bezopasnost_mult',
'audiosistema_mult',
'shini_i_diski_mult',
'electroprivod_mult',
'fary_mult',
'multimedia_navigacia_mult',
'obogrev_mult',
'pamyat_nastroek_mult',
'podushki_bezopasnosti_mult',
'pomosh_pri_vozhdenii_mult',
'protivoygonnaya_sistema_mult',
'salon_mult',
'upravlenie_klimatom_mult']:
            print(column)
            
            # Собираем все уникальные значения из всех списков (исключая None)
            unique_values = set()
            for item_list in df[column]:
                if isinstance(item_list, np.ndarray):
                    for item in item_list:
                        if item is not None:
                            unique_values.add(item)
            
            # Создаем dummy-колонки для каждого уникального значения
            for value in unique_values:
                dummy_col_name = f"{column}_{value}"
                
                # Создаем dummy-колонку: 1 если значение есть в списке, 0 если нет
                df_result[dummy_col_name] = df[column].apply(
                    lambda x: 1 if isinstance(x, np.ndarray) and value in x else 0
                )
            d_cols.append(column)
    df_result.drop(columns = d_cols,inplace = True)
    
    return df_result

df = df.fillna('No')

df = create_dummy_columns(train_df)

test_df = test_df.fillna('No')

test = create_dummy_columns(test_df)
from catboost import CatBoostRegressor
df['price_TARGET'] = np.log(df['price_TARGET'])

cat_features = ['owners_count', 'equipment', 'body_type', 'drive_type', 'engine_type',
       'doors_number', 'color', 'pts', 'audiosistema', 'diski',
       'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom',
       'usilitel_rul', 'steering_wheel','crashes_count',
       'aktivnaya_bezopasnost_mult_Курсовая устойчивость',
       'aktivnaya_bezopasnost_mult_Блок. дифференциала',
       'aktivnaya_bezopasnost_mult_Экстренное торможение',
       'aktivnaya_bezopasnost_mult_Распред. тормозных усилий',
       'aktivnaya_bezopasnost_mult_Антипробуксовка',
       'aktivnaya_bezopasnost_mult_Обнаружение пешеходов',
       'aktivnaya_bezopasnost_mult_Антиблокировка тормозов',
       'audiosistema_mult_Cабвуфер',
       'shini_i_diski_mult_Зимние шины в комплекте',
       'electroprivod_mult_Задних сидений',
       'electroprivod_mult_Передних сидений', 'electroprivod_mult_Зеркал',
       'electroprivod_mult_Складывания зеркал',
       'electroprivod_mult_Рулевой колонки', 'fary_mult_Противотуманные',
       'fary_mult_Адаптивное освещение', 'fary_mult_Омыватели фар',
       'multimedia_navigacia_mult_MP3',
       'multimedia_navigacia_mult_Управление на руле',
       'multimedia_navigacia_mult_TV', 'multimedia_navigacia_mult_CD привод',
       'multimedia_navigacia_mult_Радио',
       'multimedia_navigacia_mult_GPS-навигатор',
       'multimedia_navigacia_mult_Экран', 'multimedia_navigacia_mult_USB',
       'multimedia_navigacia_mult_Bluetooth', 'multimedia_navigacia_mult_AUX',
       'obogrev_mult_Заднего стекла', 'obogrev_mult_Задних сидений',
       'obogrev_mult_Руля', 'obogrev_mult_Передних сидений',
       'obogrev_mult_Зеркал', 'pamyat_nastroek_mult_Задних сидений',
       'pamyat_nastroek_mult_Сиденья водителя',
       'pamyat_nastroek_mult_Рулевой колонки', 'pamyat_nastroek_mult_Зеркал',
       'podushki_bezopasnosti_mult_Фронтальная для водителя',
       'podushki_bezopasnosti_mult_Шторки',
       'podushki_bezopasnosti_mult_Боковые передние',
       'podushki_bezopasnosti_mult_Боковые задние',
       'podushki_bezopasnosti_mult_Коленные',
       'pomosh_pri_vozhdenii_mult_Контроль слепых зон',
       'pomosh_pri_vozhdenii_mult_Парктроник задний',
       'pomosh_pri_vozhdenii_mult_Монохромный экран бортового компьютера',
       'pomosh_pri_vozhdenii_mult_Автопарковщик',
       'pomosh_pri_vozhdenii_mult_Камера заднего вида',
       'pomosh_pri_vozhdenii_mult_Круиз-контроль',
       'pomosh_pri_vozhdenii_mult_Датчик света',
       'pomosh_pri_vozhdenii_mult_Парктроник передний',
       'pomosh_pri_vozhdenii_mult_Датчик дождя',
       'protivoygonnaya_sistema_mult_Центральный замок',
       'protivoygonnaya_sistema_mult_Сигнализация',
       'protivoygonnaya_sistema_mult_Спутник',
       'protivoygonnaya_sistema_mult_Иммобилайзер', 'salon_mult_Люк',
       'salon_mult_Кожаный руль',
       'upravlenie_klimatom_mult_Атермальное остекление',
       'upravlenie_klimatom_mult_Управление на руле']

feature_columns = [ 'equipment', 'body_type', 'drive_type', 'engine_type',
       'doors_number', 'color', 'pts', 'audiosistema', 'diski',
       'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom',
       'usilitel_rul', 'steering_wheel', 'crashes_count', 'owners_count',
       'mileage', 'latitude', 'longitude',
       'aktivnaya_bezopasnost_mult_Курсовая устойчивость',
       'aktivnaya_bezopasnost_mult_Блок. дифференциала',
       'aktivnaya_bezopasnost_mult_Экстренное торможение',
       'aktivnaya_bezopasnost_mult_Распред. тормозных усилий',
       'aktivnaya_bezopasnost_mult_Антипробуксовка',
       'aktivnaya_bezopasnost_mult_Обнаружение пешеходов',
       'aktivnaya_bezopasnost_mult_Антиблокировка тормозов',
       'audiosistema_mult_Cабвуфер',
       'shini_i_diski_mult_Зимние шины в комплекте',
       'electroprivod_mult_Задних сидений',
       'electroprivod_mult_Передних сидений', 'electroprivod_mult_Зеркал',
       'electroprivod_mult_Складывания зеркал',
       'electroprivod_mult_Рулевой колонки', 'fary_mult_Противотуманные',
       'fary_mult_Адаптивное освещение', 'fary_mult_Омыватели фар',
       'multimedia_navigacia_mult_MP3',
       'multimedia_navigacia_mult_Управление на руле',
       'multimedia_navigacia_mult_TV', 'multimedia_navigacia_mult_CD привод',
       'multimedia_navigacia_mult_Радио',
       'multimedia_navigacia_mult_GPS-навигатор',
       'multimedia_navigacia_mult_Экран', 'multimedia_navigacia_mult_USB',
       'multimedia_navigacia_mult_Bluetooth', 'multimedia_navigacia_mult_AUX',
       'obogrev_mult_Заднего стекла', 'obogrev_mult_Задних сидений',
       'obogrev_mult_Руля', 'obogrev_mult_Передних сидений',
       'obogrev_mult_Зеркал', 'pamyat_nastroek_mult_Задних сидений',
       'pamyat_nastroek_mult_Сиденья водителя',
       'pamyat_nastroek_mult_Рулевой колонки', 'pamyat_nastroek_mult_Зеркал',
       'podushki_bezopasnosti_mult_Фронтальная для водителя',
       'podushki_bezopasnosti_mult_Шторки',
       'podushki_bezopasnosti_mult_Боковые передние',
       'podushki_bezopasnosti_mult_Боковые задние',
       'podushki_bezopasnosti_mult_Коленные',
       'pomosh_pri_vozhdenii_mult_Контроль слепых зон',
       'pomosh_pri_vozhdenii_mult_Парктроник задний',
       'pomosh_pri_vozhdenii_mult_Монохромный экран бортового компьютера',
       'pomosh_pri_vozhdenii_mult_Автопарковщик',
       'pomosh_pri_vozhdenii_mult_Камера заднего вида',
       'pomosh_pri_vozhdenii_mult_Круиз-контроль',
       'pomosh_pri_vozhdenii_mult_Датчик света',
       'pomosh_pri_vozhdenii_mult_Парктроник передний',
       'pomosh_pri_vozhdenii_mult_Датчик дождя',
       'protivoygonnaya_sistema_mult_Центральный замок',
       'protivoygonnaya_sistema_mult_Сигнализация',
       'protivoygonnaya_sistema_mult_Спутник',
       'protivoygonnaya_sistema_mult_Иммобилайзер', 'salon_mult_Люк',
       'salon_mult_Кожаный руль',
       'upravlenie_klimatom_mult_Атермальное остекление',
       'upravlenie_klimatom_mult_Управление на руле']

target_column = 'price_TARGET'

def preprocess_data(df, target_column):
    """
    Предобработка данных: обработка None и пропущенных значений
    """
    df_processed = df.copy()
    
    # Заменяем None на np.nan
    df_processed = df_processed.replace([None, 'None', 'null', 'NULL'], np.nan)
    
    # Для числовых колонок заполняем медианой
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col != target_column:  # Не заполняем таргет
            df_processed[col] = df_processed[col].fillna(df_processed[col].median())
    
    # Для категориальных колонок заполняем модой
    categorical_cols = df_processed.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_processed[col] = df_processed[col].fillna(df_processed[col].mode()[0] if not df_processed[col].mode().empty else 'missing')
    
    # Для таргета удаляем строки с пропусками
    if target_column in df_processed.columns:
        df_processed = df_processed.dropna(subset=[target_column])
    
    return df_processed
df = preprocess_data(df, target_column)

test = preprocess_data(test, target_column)

from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

def time_based_train_test_split(df, test_size=0.1):
    
    # Определяем индекс разделения
    split_idx = int(len(df) * (1 - test_size))
    
    # Разделяем данные
    train_df = df.iloc[:split_idx]
    test_df = df.iloc[split_idx:]
    
    return train_df, test_df


train_df, test_df = time_based_train_test_split(df, 0.1)
    
print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")
    
X_train = train_df[feature_columns]
y_train = train_df[target_column]
X_test = test_df[feature_columns]
y_test = test_df[target_column]
    
    # Инициализация CatBoost с оптимизацией MAPE
model = CatBoostRegressor(
        loss_function='MAPE',  # Оптимизируем MAPE напрямую
        iterations=2000,
        random_state=42,
        verbose=100,  # Выводим прогресс каждые 100 итераций
        cat_features=cat_features  # Автоматическое определение категориальных features
)
    
model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=50,
        use_best_model=True
)
    
    # Предсказания
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
    
    # Метрики
mape_train = mean_absolute_percentage_error(y_train, y_pred_train) * 100
mape_test = mean_absolute_percentage_error(y_test, y_pred_test) * 100
    
print(f"\nРезультаты:")
print(f"MAPE на train: {mape_train:.2f}%")
print(f"MAPE на test: {mape_test:.2f}%")
    

mape_train = mean_absolute_percentage_error(np.exp(y_train), np.exp(y_pred_train))* 100
mape_test = mean_absolute_percentage_error(np.exp(y_test), np.exp(y_pred_test)) * 100
    
print(f"\nРезультаты:")
print(f"MAPE на train: {mape_train:.2f}%")
print(f"MAPE на test: {mape_test:.2f}%")
    

X_train = df[feature_columns]
y_train = df[target_column]
    
    # Инициализация CatBoost с оптимизацией MAPE
model = CatBoostRegressor(
        loss_function='MAPE',  # Оптимизируем MAPE напрямую
        iterations=1500,
        random_state=42,
        verbose=100,  # Выводим прогресс каждые 100 итераций
        cat_features=cat_features  # Автоматическое определение категориальных features
)
    
model.fit(
        X_train, y_train,
        early_stopping_rounds=50,
        use_best_model=True
)
    

def median_ape(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true > 0
    ape = np.abs(y_true[mask] - y_pred[mask]) / (y_true[mask] + 1e-6)
    return float(np.median(ape))

1 / (1 + 0.44)

In [ ]:
# Инференс на тестовых данных

test_df = pd.read_csv(TEST_CSV)
test_ds = CarsDataset(test_df, is_train=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=8, pin_memory=True, drop_last=False,
                         collate_fn=collate_fn)

# Загружаем лучшую сохраненную модель
ckpt = torch.load(CKPT_PATH, map_location=device)
model.load_state_dict(ckpt["model"])
model.eval() # Переводим модель в режим предсказания

preds_all, ids_all = [], []
# torch.no_grad() отключает расчет градиентов, что ускоряет инференс и экономит память
with torch.no_grad():
    for imgs, ids in tqdm(test_loader, desc="Test", leave=False):
        imgs = imgs.to(device, non_blocking=True)
        preds = model(imgs) # Получаем предсказания в лог-масштабе

        # Если нужно, возвращаем в исходный масштаб
        if USE_LOG_TARGET:
            preds = torch.expm1(preds)

        preds_all.append(preds.cpu())
        ids_all.append(ids.cpu())

# Собираем все предсказания и ID в единые массивы
preds_all = torch.cat(preds_all).squeeze().numpy()
ids_all   = torch.cat(ids_all).squeeze().numpy()

# Формирование файла для сабмита
submission_df = pd.DataFrame({
    "ID": ids_all,
    "price_TARGET": preds_all
})

submission_df.to_csv("submission.csv", index=False)
submission_df.head()

In [ ]:
# Убедимся, что наш файл имеет ту же структуру и размер, что и образец
# Это простой, но очень полезный санити-чек

sample = pd.read_csv("sample_submission.csv")
assert list(submission_df.columns) == ["ID", "target"]
assert len(submission_df) == len(sample)

print("Размер сабмита:", submission_df.shape)